In [4]:
import gumpy as gp
import numpy as np
import pywt

#绘制confusion matrix heatmap
import seaborn as sns

import sklearn
import os

import warnings

warnings.simplefilter('ignore') #忽略警告

In [5]:
import scipy
import scipy.io as sio

from scipy import linalg

import pandas as pd

#分类器
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.lda import LDA
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

import xgboost
import lightgbm

#模型集成
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from mlxtend.classifier import StackingClassifier

#模型调节
from sklearn.model_selection import GridSearchCV #参数搜索
from mlxtend.feature_selection import SequentialFeatureSelector #特征选择函数 选择合适的feature

#结果可视化
from sklearn.metrics import classification_report , confusion_matrix #混淆矩阵

#相关指标
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import label_binarize

#二分类其多分类化
#from sklearn.multiclass import OneVsOneClassifier
#from sklearn.multiclass import OneVsRestClassifier

#from sklearn.preprocessing import StandardScaler
#from sklearn.cluster import KMeans

#距离函数 度量向量距离
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import cosine_similarity #余弦相似度

#one-hot使用
#from keras.utils import to_categorical

#绘图
import matplotlib.pyplot as plt

import scipy.linalg as la

import gc

%matplotlib inline

In [6]:
sample_rate = 256 #hz
origin_channel = 16 #5 channel eeg

#采集的通道
#共16 channel
#未使用的channel使用none代替
#reference:a study on performance increasing in ssvep based bci application
SAMPLE_CHANNEL = ['Pz' , 'PO3' , 'PO4' , 'O1' , 'O2' , 'Oz' , 'O9' , 'FP2' ,
                  'C4' , 'C6' , 'CP3' , 'CP1' ,
                  'CPZ' , 'CP2' , 'CP4' , 'PO8']

LABEL2STR = {0:'sen' , 1:'hong' , 2:'zhao',
             3:'fen' , 4:'xiao' , 5:'yu' , 
             6:'bin' , 7:'wang' , 8:'wei' , 
             9:'fei'}

# 减去前多少秒数据 second
# 减去后多少秒数据 second
CLIP_FORWARD = 2
CLIP_BACKWARD = 1

# 单个小段的实验时长
trial_time = 3 #second

trial_offset = 0 #second
start_trial_time = 0 #真正的实验开始时刻
end_trial_time = 2 #真正的实验结束时刻(<trial_time)

#是否进行归一化
#reference:a study on performance increasing in ssvep based bci application
#IS_NORMALIZE = True

#是否进行滤波
#IS_FILTER = False
#EEG频率范围
#reference:a study on performance increasing in ssvep based bci application
LO_FREQ = 0.5
HI_FREQ = 40

#是否陷波
#IS_NOTCH = False
NOTCH_FREQ = 50 #陷波 工频



# load data step

In [7]:
# defined

#def butter_worth(data , lowcut , highcut , order=6):
#    nyq = 0.5 * sample_rate
#    
#    lo = lowcut / nyq
#    hi = highcut / nyq
#    
#    b,a = scipy.signal.butter(order , [lo , hi] , btype='bandpass')
#
#    return np.array([scipy.signal.filtfilt(b , a , data[: , i]) for i in range(data.shape[1])]).reshape((-1 , origin_channel))

In [8]:
def load_data(filename):
    
    #extra_overlap = 1500
    
    data = sio.loadmat(file_name=filename)['data_received'] #length*16 matrix

    #此通道没有采集 置为0
    #全通道均使用时 不需要
    #for i in range(len(SAMPLE_CHANNEL)):
    #    if SAMPLE_CHANNEL[i] == 'none':
    #        data[: , i] = 0.0

    #删除前x秒和后x秒数据
    
    
    #是否进行裁剪 【如果进行裁剪 由于sen的第一次数据 将extra_overlap调整为1500】
    data = data[CLIP_FORWARD * sample_rate : - CLIP_BACKWARD * sample_rate]
    
    
    #data = np.concatenate((data , data[ -extra_overlap : , :]) , axis=0)
    #
    #data_filter = butter_worth(data , 0.5 , 40 , order=3)
    #
    #return data_filter[extra_overlap : , :] #将边界效应去掉

    return data

In [9]:
def separate(data , label , overlap_length = 128):
    '''
    最长重叠长度为size长 256*3 个数据点
    '''
    train_data = []
    train_labels = []

    size = sample_rate * trial_time #一小段 256*3 个数据点
    data_length = data.shape[0]

    idx = 0

    while idx<data_length-size:
        train_data.append(data[idx : idx+size , :])
        train_labels.append(label)

        idx = idx + (size - overlap_length)

    return np.array(train_data) , np.array(train_labels)

In [10]:
def train_val(data , ratio = 0.9):
    '''
    将数据分为 训练集 和 验证集
    '''
    
    seg = int(ratio * data.shape[0])
    
    return data[ : seg] , data[seg : ]

def shuffle_t_v(filenames):
    np.random.shuffle(filenames)
    
    return filenames

def combine(freq = 10):
    '''
    训练数据与验证数据
    :freq: 指定闪烁的频率
    
    '''
    
    if freq not in [10 , 15 , 20 , 25]:
        print('freq must in 10,15,20,25')
        return 
    
    ratio = 0.9 #训练集的占比
    overlap_length = 2*256 #重叠2秒数据
    
    #保证随机性 进行置乱
    person_0_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/0/%s/' % freq) )
    person_1_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/1/%s/' % freq) )
    person_2_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/2/%s/' % freq) )
    person_3_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/3/%s/' % freq) )
    person_4_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/4/%s/' % freq) )
    person_5_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/5/%s/' % freq) )
    person_6_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/6/%s/' % freq) )
    person_7_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/7/%s/' % freq) )
    person_8_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/8/%s/' % freq) )
    person_9_filenames = shuffle_t_v( os.listdir('real_data/eeg_final/circle/9/%s/' % freq) )
    

    #打开信号文件 并 合并
    person_0 = np.concatenate([load_data('real_data/eeg_final/circle/0/%s/' % freq + filename) for filename in person_0_filenames] , axis = 0)
    person_1 = np.concatenate([load_data('real_data/eeg_final/circle/1/%s/' % freq + filename) for filename in person_1_filenames] , axis = 0)
    person_2 = np.concatenate([load_data('real_data/eeg_final/circle/2/%s/' % freq + filename) for filename in person_2_filenames] , axis = 0)
    person_3 = np.concatenate([load_data('real_data/eeg_final/circle/3/%s/' % freq + filename) for filename in person_3_filenames] , axis = 0)
    person_4 = np.concatenate([load_data('real_data/eeg_final/circle/4/%s/' % freq + filename) for filename in person_4_filenames] , axis = 0)
    person_5 = np.concatenate([load_data('real_data/eeg_final/circle/5/%s/' % freq + filename) for filename in person_5_filenames] , axis = 0)
    person_6 = np.concatenate([load_data('real_data/eeg_final/circle/6/%s/' % freq + filename) for filename in person_6_filenames] , axis = 0)
    person_7 = np.concatenate([load_data('real_data/eeg_final/circle/7/%s/' % freq + filename) for filename in person_7_filenames] , axis = 0)
    person_8 = np.concatenate([load_data('real_data/eeg_final/circle/8/%s/' % freq + filename) for filename in person_8_filenames] , axis = 0)
    person_9 = np.concatenate([load_data('real_data/eeg_final/circle/9/%s/' % freq + filename) for filename in person_9_filenames] , axis = 0)
    
    person_0_train , person_0_val = train_val(person_0)
    person_1_train , person_1_val = train_val(person_1)
    person_2_train , person_2_val = train_val(person_2)
    person_3_train , person_3_val = train_val(person_3)
    person_4_train , person_4_val = train_val(person_4)
    person_5_train , person_5_val = train_val(person_5)
    person_6_train , person_6_val = train_val(person_6)
    person_7_train , person_7_val = train_val(person_7)
    person_8_train , person_8_val = train_val(person_8)
    person_9_train , person_9_val = train_val(person_9)
    
    #数据分段阶段
    
    #============
    #训练数据分段
    train_person_data_0 , train_person_labels_0 = separate(person_0_train , label = 0 , overlap_length=overlap_length)
    train_person_data_1 , train_person_labels_1 = separate(person_1_train , label = 1 , overlap_length=overlap_length)
    train_person_data_2 , train_person_labels_2 = separate(person_2_train , label = 2 , overlap_length=overlap_length)
    train_person_data_3 , train_person_labels_3 = separate(person_3_train , label = 3 , overlap_length=overlap_length)
    train_person_data_4 , train_person_labels_4 = separate(person_4_train , label = 4 , overlap_length=overlap_length)
    train_person_data_5 , train_person_labels_5 = separate(person_5_train , label = 5 , overlap_length=overlap_length)
    train_person_data_6 , train_person_labels_6 = separate(person_6_train , label = 6 , overlap_length=overlap_length)
    train_person_data_7 , train_person_labels_7 = separate(person_7_train , label = 7 , overlap_length=overlap_length)
    train_person_data_8 , train_person_labels_8 = separate(person_8_train , label = 8 , overlap_length=overlap_length)
    train_person_data_9 , train_person_labels_9 = separate(person_9_train , label = 9 , overlap_length=overlap_length)

    #合并数据
    train_data = np.concatenate((train_person_data_0 , train_person_data_1 , train_person_data_2 ,
                                 train_person_data_3 , train_person_data_4 , train_person_data_5 ,
                                 train_person_data_6 , train_person_data_7 , train_person_data_8 ,
                                 train_person_data_9 ))
    
    train_labels = np.concatenate((train_person_labels_0 , train_person_labels_1 , train_person_labels_2 ,
                                   train_person_labels_3 , train_person_labels_4 , train_person_labels_5 ,
                                   train_person_labels_6 , train_person_labels_7 , train_person_labels_8 ,
                                   train_person_labels_9 ))
    
    #产生索引并置乱
    idx_train_data = list(range(train_data.shape[0]))
    np.random.shuffle(idx_train_data)

    #将训练数据置乱
    train_data = train_data[idx_train_data]
    train_labels = train_labels[idx_train_data]
    
    #============
    #验证数据分段
    val_person_data_0 , val_person_labels_0 = separate(person_0_val , label = 0 , overlap_length=0)
    val_person_data_1 , val_person_labels_1 = separate(person_1_val , label = 1 , overlap_length=0)
    val_person_data_2 , val_person_labels_2 = separate(person_2_val , label = 2 , overlap_length=0)
    val_person_data_3 , val_person_labels_3 = separate(person_3_val , label = 3 , overlap_length=0)
    val_person_data_4 , val_person_labels_4 = separate(person_4_val , label = 4 , overlap_length=0)
    val_person_data_5 , val_person_labels_5 = separate(person_5_val , label = 5 , overlap_length=0)
    val_person_data_6 , val_person_labels_6 = separate(person_6_val , label = 6 , overlap_length=0)
    val_person_data_7 , val_person_labels_7 = separate(person_7_val , label = 7 , overlap_length=0)
    val_person_data_8 , val_person_labels_8 = separate(person_8_val , label = 8 , overlap_length=0)
    val_person_data_9 , val_person_labels_9 = separate(person_9_val , label = 9 , overlap_length=0)
    
    #合并数据
    val_data = np.concatenate((val_person_data_0 , val_person_data_1 , val_person_data_2 ,
                               val_person_data_3 , val_person_data_4 , val_person_data_5 ,
                               val_person_data_6 , val_person_data_7 , val_person_data_8 ,
                               val_person_data_9 ))
    
    val_labels = np.concatenate((val_person_labels_0 , val_person_labels_1 , val_person_labels_2 ,
                                 val_person_labels_3 , val_person_labels_4 , val_person_labels_5 ,
                                 val_person_labels_6 , val_person_labels_7 , val_person_labels_8 ,
                                 val_person_labels_9 ))

    #产生索引并置乱
    idx_val_data = list(range(val_data.shape[0]))
    np.random.shuffle(idx_val_data)

    #将训练数据置乱
    val_data = val_data[idx_val_data]
    val_labels = val_labels[idx_val_data]
    
    return train_data , train_labels , val_data , val_labels

In [11]:
def shuffle(train_data , train_labels , val_data , val_labels):
    #置乱一次数据
    idx_train_data = list(range(train_data.shape[0]))
    np.random.shuffle(idx_train_data)
    
    idx_val_data = list(range(val_data.shape[0]))
    np.random.shuffle(idx_val_data)
    
    return train_data[idx_train_data] , train_labels[idx_train_data] , val_data[idx_val_data] , val_labels[idx_val_data]

In [12]:
#train_X_ , train_y , val_X_ , val_y = combine(freq = 10) #10 15 20 25 hz

In [13]:
#如果没有进行前后裁剪 则输出数据会变多

#print(train_X_.shape , train_y.shape , val_X_.shape , val_y.shape)

In [14]:
def feature_extraction_dwt_meta(data , n):
    n_features = 48
    
    X = np.zeros((data.shape[0] , n_features))
    
    level = 5
    wavelet = 'db4'
    
    #n=3 or 4
    
    for i , datum in enumerate(data):
        coeffs_Pz  = pywt.wavedec(data = datum[:,0], wavelet=wavelet, level=level)
        coeffs_PO3 = pywt.wavedec(data = datum[:,1], wavelet=wavelet, level=level)
        coeffs_PO4 = pywt.wavedec(data = datum[:,2], wavelet=wavelet, level=level)
        coeffs_O1  = pywt.wavedec(data = datum[:,3], wavelet=wavelet, level=level)
        coeffs_O2  = pywt.wavedec(data = datum[:,4], wavelet=wavelet, level=level)
        coeffs_Oz  = pywt.wavedec(data = datum[:,5], wavelet=wavelet, level=level)
        coeffs_O9  = pywt.wavedec(data = datum[:,6], wavelet=wavelet, level=level)
        coeffs_FP2 = pywt.wavedec(data = datum[:,7], wavelet=wavelet, level=level)
        coeffs_C4  = pywt.wavedec(data = datum[:,8], wavelet=wavelet, level=level)
        coeffs_C6  = pywt.wavedec(data = datum[:,9], wavelet=wavelet, level=level)
        coeffs_CP3 = pywt.wavedec(data = datum[:,10], wavelet=wavelet, level=level)
        coeffs_CP1 = pywt.wavedec(data = datum[:,11], wavelet=wavelet, level=level)
        coeffs_CPZ = pywt.wavedec(data = datum[:,12], wavelet=wavelet, level=level)
        coeffs_CP2 = pywt.wavedec(data = datum[:,13], wavelet=wavelet, level=level)
        coeffs_CP4 = pywt.wavedec(data = datum[:,14], wavelet=wavelet, level=level)
        coeffs_PO8 = pywt.wavedec(data = datum[:,15], wavelet=wavelet, level=level)

        X[i , :] = np.array([
            np.std(coeffs_Pz [n]),   
            np.std(coeffs_PO3[n]),  
            np.std(coeffs_PO4[n]),   
            np.std(coeffs_O1 [n]),  
            np.std(coeffs_O2 [n]),  
            np.std(coeffs_Oz [n]),   
            np.std(coeffs_O9 [n]),  
            np.std(coeffs_FP2[n]),  
            np.std(coeffs_C4 [n]),
            np.std(coeffs_C6 [n]),
            np.std(coeffs_CP3[n]),
            np.std(coeffs_CP1[n]),
            np.std(coeffs_CPZ[n]),
            np.std(coeffs_CP2[n]),
            np.std(coeffs_CP4[n]),
            np.std(coeffs_PO8[n]),
            
            np.mean(coeffs_Pz [n]**2),
            np.mean(coeffs_PO3[n]**2),
            np.mean(coeffs_PO4[n]**2),
            np.mean(coeffs_O1 [n]**2),
            np.mean(coeffs_O2 [n]**2),
            np.mean(coeffs_Oz [n]**2),
            np.mean(coeffs_O9 [n]**2),
            np.mean(coeffs_FP2[n]**2),            
            np.mean(coeffs_C4 [n]**2),
            np.mean(coeffs_C6 [n]**2),
            np.mean(coeffs_CP3[n]**2),
            np.mean(coeffs_CP1[n]**2),
            np.mean(coeffs_CPZ[n]**2),
            np.mean(coeffs_CP2[n]**2),
            np.mean(coeffs_CP4[n]**2),
            np.mean(coeffs_PO8[n]**2),
            
            np.mean(coeffs_Pz [n]),
            np.mean(coeffs_PO3[n]), 
            np.mean(coeffs_PO4[n]),
            np.mean(coeffs_O1 [n]),
            np.mean(coeffs_O2 [n]),
            np.mean(coeffs_Oz [n]),
            np.mean(coeffs_O9 [n]),
            np.mean(coeffs_FP2[n]),        
            np.mean(coeffs_C4 [n]),
            np.mean(coeffs_C6 [n]),
            np.mean(coeffs_CP3[n]),
            np.mean(coeffs_CP1[n]),
            np.mean(coeffs_CPZ[n]),
            np.mean(coeffs_CP2[n]),
            np.mean(coeffs_CP4[n]),
            np.mean(coeffs_PO8[n])]).flatten()
        
    return X

def normalize(data , normalization_type = 'mean_std'):
    
    def _norm_mean_std(data):
        _mean = np.mean(data , axis=0)
        _std = np.std(data , axis=0)
        
        return (data - _mean) / _std
    
    def _norm_min_max(data):
        return (data - np.min(data)) / (np.max(data) - np.min(data))
    
    if normalization_type == 'mean_std':
        return _norm_mean_std(data)
    elif normalization_type == 'min_max':
        return _norm_min_max(data)
    else:
        raise Exception('wrong normalization type')
    
def feature_extraction_dwt(data , is_normalize = True):
    data_3 = feature_extraction_dwt_meta(data , 3) #4
    data_4 = feature_extraction_dwt_meta(data , 4) #5
    
    data_concat = np.concatenate((data_3 , data_4) , axis = -1)
    
    if is_normalize:
        return normalize(data_concat)
    else:
        return data_concat

In [15]:
#train_X = feature_extraction_dwt(train_X_)

#val_X = feature_extraction_dwt(val_X_)

In [16]:
#print(train_X.shape , train_y.shape , val_X.shape , val_y.shape)

In [17]:
def con_mat(_feature , _labels , model):
    '''
    打印训练结果
    '''
    
    print('val score:%f' % model.score(_feature , _labels))
    print('real')
    
    print(confusion_matrix(_labels , model.predict(_feature)))
    print(classification_report(_labels , model.predict(_feature)))
    
def con_mat_heatmap(_feature , _labels , model , color , png_path):
    
    _labels_hat = model.predict(_feature)
    
    mat = confusion_matrix( _labels , _labels_hat )
    
    sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False , cmap=color )#,
            #xticklabels=faces.target_names,
            #yticklabels=faces.target_names)
    
    precision = precision_score(_labels , _labels_hat , average='macro')
    recall = recall_score(_labels , _labels_hat , average='macro')
    f1 = f1_score(_labels , _labels_hat , average='macro')
    
    plt.xlabel('precision:%.2f recall:%.2f f1:%.2f' % (precision , recall , f1) )
    # plt.ylabel('predicted label');
    plt.savefig(png_path) #保存起来
    plt.close()

def feature_selection(data , labels , model , num_features , cv=10):
    '''
    :model: classify model
    :num_features: features count you expect(integer or tuple)
    '''
    
    '''[8 20]'''
    
    sfs = SequentialFeatureSelector(model , k_features=num_features , cv=cv , verbose = 2 , n_jobs=-1) #all cpu cores
    
    sfs.fit(data , labels)
    
    #最优秀的特征索引
    return sfs.k_feature_idx_


def choose_common_feature_idx(data , labels , classifiers , use_ratio = 0.25 , num_features = 10 , num_features_threshold = 8):
    '''
    sub_band_power使用该函数 进行筛选特征
    
    :num_features:integer or tuple 期望的特征数量（待选择的数量）
    :min_num_features: 特征数量阈值 小于时 停止选择
    选择适合所有分类器的特征索引值
    集合 与 运算
    '''
    
    idxes_ratio = np.random.randint(0 , data.shape[0] , size = int(use_ratio * data.shape[0]) )
    
    data_shuffle = data #[idxes_ratio]
    labels_shuffle = labels #[idxes_ratio]
    
    feature_idxes = set(list(range(data.shape[1]))) #初始化为所有的特征索引值
    
    #===========
    #随机使用原有索引中的 20%
    random_idxes = np.random.randint(0 , data.shape[1] , size = 25 )
    
    data_shuffle = data_shuffle[: , random_idxes]
    labels_shuffle = labels_shuffle #保持完整性
    #===========


    for classifier in classifiers:
        
        idx = feature_selection(data_shuffle , labels_shuffle , classifier , num_features)
        idx = set(idx)
        
        #寻找共同的特征索引
        #寻找之前先测试 如果小于阈值 直接停止
        if len(feature_idxes & idx) < num_features_threshold:
            break
        
        feature_idxes = feature_idxes & idx
        
        break
        
    return np.array(list(feature_idxes))

In [18]:
def f1_score_(_feature , _labels , model):
    
    _labels_hat = model.predict(_feature)
    
    accu = accuracy_score(_labels , _labels_hat )
    precision = precision_score(_labels , _labels_hat , average='macro')
    recall = recall_score(_labels , _labels_hat , average='macro')
    f1 = f1_score(_labels , _labels_hat , average='macro')
    kappa = cohen_kappa_score(_labels , _labels_hat)
    
    return accu , precision, recall , f1 , kappa

def roc_auc(real_label , score):
    return roc_auc_score( label_binarize(real_label , classes=[0,1,2,3,4,5,6,7,8,9]) , score , average='micro')

def itrr(acc , N=10):
    return np.log2(N) + acc*np.log2(acc) + (1-acc)*np.log2((1-acc)/(N-1))

In [19]:
#初始化所有分类器
lgbm = lightgbm.LGBMClassifier()
xgb = xgboost.XGBClassifier()
gbc = GradientBoostingClassifier()
rf =  RandomForestClassifier()

#svc = SVC(probability=True)
#adaboost = AdaBoostClassifier()
#knn = KNeighborsClassifier()
#dt = DecisionTreeClassifier()
#lda = LDA()
#nb = GaussianNB()
#mlp = MLPClassifier()

#基分类器
classifiers = [lgbm , gbc]

#基分类器
estimators = [('lightgbm' , lgbm) , ('xgboost' , xgb) ,  ('gradientboosting' , gbc) ,
                             ('randomforest' , rf) ]


#下面的顺序按照三篇论文与经验得出
#voting = VotingClassifier(estimators = estimators ,
#                voting='soft' , n_jobs=-1)

In [20]:
for freq in [10 , 15 , 20 , 25]:
    #for iter_classifier in iter_classifiers:
        #estimators.append(iter_classifier)
    
    for (i , classifier) in enumerate(classifiers):
        acc_s = []
        pre_s = []
        rec_s = []
        f1s_s = []
        kap_s = []
        
        auc_s = []
        
        itr_s = []
                

        for t in range(10): #循环20次 取平均值 作为最终结果
            voting = classifier

            train_X_ , train_y , val_X_ , val_y = combine(freq=freq)

            #提取特征
            train_X = feature_extraction_dwt(train_X_)
            val_X = feature_extraction_dwt(val_X_)

            voting.fit(train_X , train_y)

            acc , pre, rec , f1s , kap = f1_score_(val_X , val_y , voting)
            
            auc = roc_auc(val_y , voting.predict_proba(val_X))
            
            itr = itrr(acc)
            
            acc_s.append(acc)
            #pre_s.append(pre)
            #rec_s.append(rec)
            #f1s_s.append(f1s)
            kap_s.append(kap)
            auc_s.append(auc)
            itr_s.append(itr)
            
            print('[%.6f %.6f %.6f %.6f]' % (acc , kap , auc , itr) )
            
        print('[%d %d] %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f' % (freq , i , np.max(acc_s) , np.min(acc_s) , np.mean(acc_s) , np.var(acc_s) , np.max(kap_s) , np.min(kap_s) , np.mean(kap_s) , np.var(kap_s) ,np.max(auc_s) , np.min(auc_s) , np.mean(auc_s) , np.var(auc_s) ,np.max(itr_s) , np.min(itr_s) , np.mean(itr_s) , np.var(itr_s)) )

[0.600000 0.555556 0.911333 1.083007]
[0.600000 0.555556 0.816133 1.083007]
[0.700000 0.666667 0.908000 1.489660]
[0.500000 0.444444 0.798133 0.736966]
[0.620000 0.577778 0.934711 1.159315]
[0.680000 0.644444 0.886444 1.403171]
[0.400000 0.333333 0.811556 0.449022]
[0.720000 0.688889 0.958267 1.578898]
[0.800000 0.777778 0.960400 1.966015]
[0.500000 0.444444 0.908089 0.736966]
[10 0] 0.800000 0.400000 0.612000 0.012976 0.777778 0.333333 0.568889 0.016020 0.960400 0.798133 0.889307 0.003278 1.966015 0.449022 1.168603 0.186746
[0.200000 0.111111 0.664978 0.064060]
[0.400000 0.333333 0.886044 0.449022]
[0.320000 0.244444 0.718622 0.261998]
[0.200000 0.111111 0.727022 0.064060]
[0.500000 0.444444 0.892889 0.736966]
[0.300000 0.222222 0.856133 0.221690]
[0.300000 0.222222 0.665778 0.221690]
[0.280000 0.200000 0.732711 0.184131]
[0.360000 0.288889 0.854889 0.350493]
[0.300000 0.222222 0.679778 0.221690]
[10 1] 0.500000 0.200000 0.316000 0.007184 0.444444 0.111111 0.240000 0.008869 0.892889 0

In [21]:
for freq in [10 , 15 , 20 , 25]:
    #for iter_classifier in iter_classifiers:
        #estimators.append(iter_classifier)
    
    acc_s = []
    pre_s = []
    rec_s = []
    f1s_s = []
    kap_s = []

    auc_s = []

    itr_s = []


    for t in range(10): #循环20次 取平均值 作为最终结果
        voting = VotingClassifier(estimators=estimators , voting='soft' , n_jobs=-1)

        train_X_ , train_y , val_X_ , val_y = combine(freq=freq)

        #提取特征
        train_X = feature_extraction_dwt(train_X_)
        val_X = feature_extraction_dwt(val_X_)

        voting.fit(train_X , train_y)

        acc , pre, rec , f1s , kap = f1_score_(val_X , val_y , voting)

        auc = roc_auc(val_y , voting.predict_proba(val_X))

        itr = itrr(acc)

        acc_s.append(acc)
        #pre_s.append(pre)
        #rec_s.append(rec)
        #f1s_s.append(f1s)
        kap_s.append(kap)
        auc_s.append(auc)
        itr_s.append(itr)

        print('[%.6f %.6f %.6f %.6f]' % (acc , kap , auc , itr) )

    print('%d %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f' % (freq , np.max(acc_s) , np.min(acc_s) , np.mean(acc_s) , np.var(acc_s) , np.max(kap_s) , np.min(kap_s) , np.mean(kap_s) , np.var(kap_s) ,np.max(auc_s) , np.min(auc_s) , np.mean(auc_s) , np.var(auc_s) ,np.max(itr_s) , np.min(itr_s) , np.mean(itr_s) , np.var(itr_s)) )

[0.500000 0.444444 0.891244 0.736966]
[0.620000 0.577778 0.920933 1.159315]
[0.400000 0.333333 0.755956 0.449022]
[0.600000 0.555556 0.923600 1.083007]
[0.300000 0.222222 0.724067 0.221690]
[0.760000 0.733333 0.927222 1.766106]
[0.600000 0.555556 0.899422 1.083007]
[0.380000 0.311111 0.829022 0.398533]
[0.500000 0.444444 0.947378 0.736966]
[0.440000 0.377778 0.768667 0.557183]
10 0.760000 0.300000 0.510000 0.016900 0.733333 0.222222 0.455556 0.020864 0.947378 0.724067 0.858751 0.006103 1.766106 0.221690 0.819179 0.190469
[0.900000 0.888889 0.985822 2.535940]
[0.720000 0.688889 0.981689 1.578898]
[0.880000 0.866667 0.996444 2.412176]
[0.900000 0.888889 0.986622 2.535940]
[0.820000 0.800000 0.973867 2.071265]
[0.900000 0.888889 0.998000 2.535940]
[0.900000 0.888889 0.998311 2.535940]
[0.640000 0.600000 0.969378 1.238072]
[0.860000 0.844444 0.985778 2.293900]
[0.800000 0.777778 0.979111 1.966015]
15 0.900000 0.640000 0.832000 0.007216 0.888889 0.600000 0.813333 0.008909 0.998311 0.969378 

In [22]:
for freq in [10 , 15 , 20 , 25]:
    #for iter_classifier in iter_classifiers:
        #estimators.append(iter_classifier)
    
    acc_s = []
    pre_s = []
    rec_s = []
    f1s_s = []
    kap_s = []

    auc_s = []

    itr_s = []


    for t in range(10): #循环20次 取平均值 作为最终结果
        voting = BaggingClassifier(base_estimator=lgbm)

        train_X_ , train_y , val_X_ , val_y = combine(freq=freq)

        #提取特征
        train_X = feature_extraction_dwt(train_X_)
        val_X = feature_extraction_dwt(val_X_)

        voting.fit(train_X , train_y)

        acc , pre, rec , f1s , kap = f1_score_(val_X , val_y , voting)

        auc = roc_auc(val_y , voting.predict_proba(val_X))

        itr = itrr(acc)

        acc_s.append(acc)
        #pre_s.append(pre)
        #rec_s.append(rec)
        #f1s_s.append(f1s)
        kap_s.append(kap)
        auc_s.append(auc)
        itr_s.append(itr)

        print('[%.6f %.6f %.6f %.6f]' % (acc , kap , auc , itr) )

    print('%d %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f' % (freq , np.max(acc_s) , np.min(acc_s) , np.mean(acc_s) , np.var(acc_s) , np.max(kap_s) , np.min(kap_s) , np.mean(kap_s) , np.var(kap_s) ,np.max(auc_s) , np.min(auc_s) , np.mean(auc_s) , np.var(auc_s) ,np.max(itr_s) , np.min(itr_s) , np.mean(itr_s) , np.var(itr_s)) )

[0.800000 0.777778 0.946889 1.966015]
[0.660000 0.622222 0.918267 1.319335]
[0.700000 0.666667 0.915333 1.489660]
[0.580000 0.533333 0.909511 1.009106]
[0.560000 0.511111 0.924578 0.937574]
[0.800000 0.777778 0.912178 1.966015]
[0.400000 0.333333 0.825644 0.449022]
[0.640000 0.600000 0.914800 1.238072]
[0.600000 0.555556 0.857200 1.083007]
[0.720000 0.688889 0.886800 1.578898]
10 0.800000 0.400000 0.646000 0.013044 0.777778 0.333333 0.606667 0.016104 0.946889 0.825644 0.901120 0.001135 1.966015 0.449022 1.303670 0.199220
[0.800000 0.777778 0.968844 1.966015]
[0.800000 0.777778 0.992267 1.966015]
[0.820000 0.800000 0.984889 2.071265]
[0.900000 0.888889 0.982844 2.535940]
[0.800000 0.777778 0.987422 1.966015]
[0.700000 0.666667 0.978711 1.489660]
[0.840000 0.822222 0.987289 2.180431]
[0.900000 0.888889 0.984711 2.535940]
[0.900000 0.888889 0.991333 2.535940]
[0.800000 0.777778 0.990400 1.966015]
15 0.900000 0.700000 0.826000 0.003524 0.888889 0.666667 0.806667 0.004351 0.992267 0.968844 

In [23]:
for freq in [10 , 15 , 20 , 25]:
    #for iter_classifier in iter_classifiers:
        #estimators.append(iter_classifier)
    
    acc_s = []
    pre_s = []
    rec_s = []
    f1s_s = []
    kap_s = []

    auc_s = []

    itr_s = []


    for t in range(10): #循环20次 取平均值 作为最终结果
        voting = StackingClassifier(classifiers=[xgb , gbc , rf] , meta_classifier=lgbm)

        train_X_ , train_y , val_X_ , val_y = combine(freq=freq)

        #提取特征
        train_X = feature_extraction_dwt(train_X_)
        val_X = feature_extraction_dwt(val_X_)

        voting.fit(train_X , train_y)

        acc , pre, rec , f1s , kap = f1_score_(val_X , val_y , voting)

        auc = roc_auc(val_y , voting.predict_proba(val_X))

        itr = itrr(acc)

        acc_s.append(acc)
        #pre_s.append(pre)
        #rec_s.append(rec)
        #f1s_s.append(f1s)
        kap_s.append(kap)
        auc_s.append(auc)
        itr_s.append(itr)

        print('[%.6f %.6f %.6f %.6f]' % (acc , kap , auc , itr) )

    print('%d %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f %.6f' % (freq , np.max(acc_s) , np.min(acc_s) , np.mean(acc_s) , np.var(acc_s) , np.max(kap_s) , np.min(kap_s) , np.mean(kap_s) , np.var(kap_s) ,np.max(auc_s) , np.min(auc_s) , np.mean(auc_s) , np.var(auc_s) ,np.max(itr_s) , np.min(itr_s) , np.mean(itr_s) , np.var(itr_s)) )

[0.800000 0.777778 0.927222 1.966015]
[0.500000 0.444444 0.781800 0.736966]
[0.700000 0.666667 0.900000 1.489660]
[0.700000 0.666667 0.882333 1.489660]
[0.600000 0.555556 0.801044 1.083007]
[0.640000 0.600000 0.871733 1.238072]
[0.760000 0.733333 0.922689 1.766106]
[0.720000 0.688889 0.917978 1.578898]
[0.780000 0.755556 0.912000 1.864377]
[0.700000 0.666667 0.909956 1.489660]
10 0.800000 0.500000 0.690000 0.007300 0.777778 0.444444 0.655556 0.009012 0.927222 0.781800 0.882676 0.002370 1.966015 0.736966 1.470242 0.124315
[0.900000 0.888889 0.993333 2.535940]
[0.900000 0.888889 0.956667 2.535940]
[0.780000 0.755556 0.813400 1.864377]
[0.900000 0.888889 0.993333 2.535940]
[0.800000 0.777778 0.946111 1.966015]
[0.880000 0.866667 0.990711 2.412176]
[0.800000 0.777778 0.919444 1.966015]
[0.700000 0.666667 0.853333 1.489660]
[0.900000 0.888889 0.956667 2.535940]
[0.800000 0.777778 0.830000 1.966015]
15 0.900000 0.700000 0.836000 0.004384 0.888889 0.666667 0.817778 0.005412 0.993333 0.813400 

In [18]:
#chosen_idx = choose_common_feature_idx()

In [19]:
#*****************
#*****************
#chosen_idx=np.array(list(range(train_X.shape[1])))
#*****************
#*****************

In [20]:
#train_X_chosen = train_X[: , chosen_idx]

In [21]:
#train_X_chosen.shape

In [22]:
#voting = VotingClassifier(estimators=[('xgboost' , xgb) , ('lightgbm' , lgbm) , ('gradientboosting' , gbc) ,
#                             ('randomforest' , rf) , ('svm' , svc) , ('decisiontree' , dt) , ('lda' , lda) ,
#                             ('adaboost' , adaboost) , ('mlp' , mlp) , ('naive bayes' , nb) , ('kneighbors' , knn)] ,
#                voting='soft' , n_jobs=-1)

In [23]:
#voting.fit(train_X_chosen , train_y)

In [24]:
#val_X_chosen = val_X[: , chosen_idx]

In [25]:
#con_mat_heatmap(val_X_chosen , val_y , voting , color=None , png_path='ttt.png')

In [26]:
'''
for freq in [10 , 15 , 20 , 25]:
    for t in range(20): #每种频率随机测试 20 次
        #读取数据
        train_X_ , train_y , val_X_ , val_y = combine(freq=freq) #10 15 20 25 hz

        #提取特征
        train_X = feature_extraction_dwt(train_X_)
        val_X = feature_extraction_dwt(val_X_)

        #初始化所有分类器
        svc = SVC(probability=True)
        rf =  RandomForestClassifier()
        adaboost = AdaBoostClassifier()
        xgb = xgboost.XGBClassifier()
        lgbm = lightgbm.LGBMClassifier()
        gbc = GradientBoostingClassifier()
        knn = KNeighborsClassifier()
        dt = DecisionTreeClassifier()
        lda = LDA()
        nb = GaussianNB()
        mlp = MLPClassifier()

        classifiers = [xgb , lgbm , gbc , rf , svc , dt , lda , adaboost , mlp , nb , knn ]

        voting = VotingClassifier(estimators=[('xgboost' , xgb) , ('lightgbm' , lgbm) , ('gradientboosting' , gbc) ,
                                 ('randomforest' , rf) , ('svm' , svc) , ('decisiontree' , dt) , ('lda' , lda) ,
                                 ('adaboost' , adaboost) , ('mlp' , mlp) , ('naive bayes' , nb) , ('kneighbors' , knn)] ,
                    voting='soft' , n_jobs=-1)

        #特征选择
        chosen_idx = choose_common_feature_idx(train_X , train_y , classifiers=classifiers , use_ratio=0.25)

        #chosen_idx=np.array(list(range(train_X.shape[1])))

        train_X_chosen = train_X[: , chosen_idx]    

        voting.fit(train_X_chosen , train_y)    
        
        val_X_chosen = val_X[: , chosen_idx]

        con_mat_heatmap(val_X_chosen , val_y , voting , color=None ,
                        png_path = 'c://Users/qq122/Desktop/v1_paper_pic/DWT/有特征选择/%shz/%s.png' % (freq , t+1) )
                        
                        '''

"\nfor freq in [10 , 15 , 20 , 25]:\n    for t in range(20): #每种频率随机测试 20 次\n        #读取数据\n        train_X_ , train_y , val_X_ , val_y = combine(freq=freq) #10 15 20 25 hz\n\n        #提取特征\n        train_X = feature_extraction_dwt(train_X_)\n        val_X = feature_extraction_dwt(val_X_)\n\n        #初始化所有分类器\n        svc = SVC(probability=True)\n        rf =  RandomForestClassifier()\n        adaboost = AdaBoostClassifier()\n        xgb = xgboost.XGBClassifier()\n        lgbm = lightgbm.LGBMClassifier()\n        gbc = GradientBoostingClassifier()\n        knn = KNeighborsClassifier()\n        dt = DecisionTreeClassifier()\n        lda = LDA()\n        nb = GaussianNB()\n        mlp = MLPClassifier()\n\n        classifiers = [xgb , lgbm , gbc , rf , svc , dt , lda , adaboost , mlp , nb , knn ]\n\n        voting = VotingClassifier(estimators=[('xgboost' , xgb) , ('lightgbm' , lgbm) , ('gradientboosting' , gbc) ,\n                                 ('randomforest' , rf) , ('svm' , svc) , (